<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#还原" data-toc-modified-id="还原-1">还原</a></span></li><li><span><a href="#编译" data-toc-modified-id="编译-2">编译</a></span></li><li><span><a href="#验证" data-toc-modified-id="验证-3">验证</a></span></li></ul></div>

# <div align='center'> 将编译后的pb2文件转换为proto文件</div>

In [6]:
import os
import sys
from google.protobuf.descriptor import FieldDescriptor as FD
from k12libs.utils.nb_easy import k12ai_get_app_dir
from k12libs.utils.nb_easy import K12AI_NBDATA_ROOT

In [7]:
proto_outdir = os.path.join(K12AI_NBDATA_ROOT, 'vulkan/protos/layers')
if not os.path.exists(proto_outdir):
    os.makedirs(proto_outdir)
sys.path.append(k12ai_get_app_dir('cv'))
sys.path.append(K12AI_NBDATA_ROOT)

## 还原

In [8]:
from vulkan.protos.layers.conv_pb2 import ConvLayer
from vulkan.protos.layers.activations_pb2 import ActLayer
from vulkan.protos.layers.dropout_pb2 import DropoutLayer
from vulkan.protos.layers.linear_pb2 import LinearLayer
from vulkan.protos.layers.padding_pb2 import PaddingLayer
from vulkan.protos.layers.pooling_pb2 import PoolLayer
from vulkan.protos.layers.norm_pb2 import NormLayer
from vulkan.protos.layers.basefunc_pb2 import BaseFuncLayer
from vulkan.protos.layers.vulkan_pb2 import VulkanLayer
from vulkan.protos.layers.layers_pb2 import Layer
from vulkan.protos.skeletons.skeleton_pb2 import skeletons

classes = [
    ConvLayer,
    ActLayer,
    DropoutLayer,
    LinearLayer,
    PaddingLayer,
    PoolLayer,
    NormLayer,
    BaseFuncLayer,
    VulkanLayer
]

ModuleNotFoundError: No module named 'vulkan.protos.layers.layers_pb2'

In [ ]:
type2string = {
    FD.TYPE_BOOL: 'bool',
    FD.TYPE_BYTES: 'bytes',
    FD.TYPE_DOUBLE: 'double',
    FD.TYPE_ENUM: 'enum',
    FD.TYPE_FIXED32: 'fixed32',
    FD.TYPE_FIXED64: 'fixed64',
    FD.TYPE_FLOAT: 'float',
    FD.TYPE_GROUP: 'group',
    FD.TYPE_INT32: 'int32',
    FD.TYPE_INT64: 'int64',
    FD.TYPE_SFIXED32: 'sfixed32',
    FD.TYPE_SFIXED64: 'sfixed64',
    FD.TYPE_SINT32: 'sint32',
    FD.TYPE_SINT64: 'sint64',
    FD.TYPE_STRING: 'string',
    FD.TYPE_UINT32: 'uint32',
    FD.TYPE_UINT64: 'uint64'
}

def class2proto(cls, space=0):
    obj = cls()
    proto = '%smessage %s {\n' % (' ' * space, cls.__name__)
    space += 4
    for field in obj.DESCRIPTOR.fields:
        # print(field.name, field.id, field.index, field.label, field.number,
        #         field.type, field.message_type, field.cpp_type, field.enum_type,
        #         field.has_options, field.has_default_value, field.default_value)

        msg_type = field.message_type

        if field.type == FD.TYPE_MESSAGE:
            proto += class2proto(msg_type._concrete_class, space)
            str_type = msg_type._concrete_class.__name__
        elif field.type == FD.TYPE_ENUM:
            str_type = field.enum_type.name
            proto += '%senum %s {\n' % (' ' * space, str_type)
            for idx, val in enumerate(field.enum_type.values):
                proto += f'{" "*(space + 4)}{val.name} = {idx};\n'
            proto += '%s}\n' % (' ' * space)
            default_value = field.enum_type.values[field.default_value].name
        else:
            str_type = type2string[field.type]
            default_value = field.default_value
            
        default = ''
        if field.has_default_value:
            default = f' [default = {default_value}]'

        label = 'required'
        if field.label == FD.LABEL_REPEATED:
            label = 'repeated'
        elif field.label == FD.LABEL_OPTIONAL:
            label = 'optional'

        proto += f'{" "*space}{label} {str_type} {field.name} = {field.number}{default};\n'
    proto += '%s}\n' % (' ' * (space - 4))
    return proto

In [ ]:
for cls in classes:
    print('-'*20, cls.DESCRIPTOR.file.name, '-'*20)
    protostr = 'syntax = "proto2";\n\n{}'.format(class2proto(cls))
    print(protostr)
    proto_file = os.path.basename(cls.DESCRIPTOR.file.name)
    with open(os.path.join(proto_outdir, proto_file), 'w') as fw:
        fw.write(protostr)

In [ ]:
# print(class2proto(Layer))

In [ ]:
# print(class2proto(skeletons))

----------------------------

## 编译

In [ ]:
from k12libs.utils.nb_easy import k12ai_restart_kernel
k12ai_restart_kernel()

In [ ]:
import sys
from k12libs.utils.nb_easy import K12AI_NBDATA_ROOT

sys.path.append(K12AI_NBDATA_ROOT)

In [ ]:
!apt install protobuf-compiler -y
!cd {K12AI_NBDATA_ROOT}; protoc --python_out=.  vulkan/protos/layers/*.proto

-----------------------

## 验证

In [ ]:
from vulkan.protos.layers.conv_pb2 import ConvLayer
from vulkan.protos.layers.activations_pb2 import ActLayer
from vulkan.protos.layers.dropout_pb2 import DropoutLayer
from vulkan.protos.layers.linear_pb2 import LinearLayer
from vulkan.protos.layers.padding_pb2 import PaddingLayer
from vulkan.protos.layers.pooling_pb2 import PoolLayer
from vulkan.protos.layers.norm_pb2 import NormLayer
from vulkan.protos.layers.basefunc_pb2 import BaseFuncLayer
from vulkan.protos.layers.vulkan_pb2 import VulkanLayer
from google.protobuf import text_format

layers_def = {
"conv": (ConvLayer, r"""
    name : "conv1"
    layer_mode : CONV2D 
    inputs : "layer1"
    outputs : "layer3"
    layer_params {
        in_channels : "3"
        out_channels : "6"
        kernel_size : "2"
        kernel_size : "3"
        bias : "True"
    }
    layer_builder : "NNTorchLayer"
    """),
"pool": (PoolLayer, r"""
    name : "pool1"
    layer_mode : MAXPOOL2D
    inputs : "layer2"
    outputs : "layer3"
    layer_params {
        kernel_size : "2"
        kernel_size : "2"
        padding : "1"
        return_indices : "False"
    }
    layer_builder : "NNTorchLayer"
    """),
"linear": (LinearLayer, r"""
    name : "linear1"
    inputs : "layer1"
    outputs : "layer2"
    layer_mode : LINEAR
    layer_params {
        in_features : "20"
        out_features : "30"
        bias : "False"
    }
    layer_builder : "NNTorchLayer"
    """),
"dropout": (DropoutLayer, r"""
    name : "dropout1"
    inputs : "layer1"
    outputs : "layer2"
    layer_mode : DROPOUT
    layer_params {
        inplace : "True"
        p : "0.6"
    }
    layer_builder : "NNTorchLayer"
    """),
"relu": (ActLayer, r"""
    name : "relu1"
    inputs : "layer1"
    outputs : "layer2"
    layer_mode : RELU
    layer_params {
        inplace : "False"
    }
    layer_builder : "NNTorchLayer"
    """),
"reshape": (VulkanLayer, r"""
    name : "reshape1"
    inputs : "layer1"
    layer_mode : RESHAPE
    layer_params {
        target_shape : "4096"
    }
    layer_builder : "NNTorchLayer"
    """),
"cat": (BaseFuncLayer, r"""
    name : "cat1"
    inputs : "layer1"
    inputs : "layer2"
    layer_mode : CAT
    layer_params {
        dim : "0"
    }
    layer_builder : "TorchFuncLayer"
    """),
"norm": (NormLayer, r"""
    name : "bn1"
    inputs : "layer1"
    outputs : "layer2"
    layer_mode : BATCHNORM2D
    layer_params {
        num_features : "36"
    }
    layer_builder : "NNTorchLayer"
    """),
"flatten": (VulkanLayer, r"""
    name : "flatten1"
    inputs : "layer1"
    outputs : "layer2"
    layer_mode : FLATTEN
    layer_params : {
        start_dim : "1"
        end_dim : "-1"
    }
    layer_builder : "NNTorchLayer"
    """),
"add": (BaseFuncLayer, r"""
    name : "add1"
    inputs : "layer1"
    inputs : "layer2"
    layer_mode : ADD
    layer_builder : "TorchFuncLayer"
    """),
"padding": (PaddingLayer, r"""
    name : "pad1"
    inputs : "layer1"
    layer_mode : ZEROPAD2D
    layer_params : {
        padding : "1"
        padding : "1"
    }
    layer_builder : "NNTorchLayer"
    """)
}

In [ ]:
for layer, (cls, netdef) in layers_def.items():
    print('-'*20, cls.DESCRIPTOR.file.name, '-'*20)
    print(netdef)
    proto = text_format.Merge(netdef, cls())
    print('-'*10, layer, '-'*10, '\n')
    print(proto)